In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>

<i>The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.</i>''')

# Climate change and radiative forcing
You probably read that methane is a greenhouse gas more potent than CO<sub>2</sub>. What
does this mean? To what extent is it true?

This is a model of greenhouse gases' radiative forcing, using the values of the
IPCC AR5, pages 731-738.

    https://www.ipcc.ch/pdf/assessment-report/ar5/wg1/WG1AR5_Chapter08_FINAL.pdf

CO2 follows a more intricate cycle, and cannot be modelled with a simple decay
function. The model used can be found in Joos et al. (2013).
    
    https://www.atmos-chem-phys.net/13/2793/2013/acp-13-2793-2013.pdf

This is inspired by the PhD thesis of Allan Shimako, available here
    
    https://tel.archives-ouvertes.fr/tel-01706684/document

Radiative forcing is the ability of certain gases to "trap" heat in the atmosphere,
this phenomenon is better known as greenhouse effect. Some gases are more efficient
than others at trapping heat, and some remain much longer than others.

The output of this code is an animation with three panels, showing the effect of
the instant release of 1 Mt (1 megatonne = 1 million tonnes = 1 billion kilograms)
of each of the greenhouse gases selected, where:
- the top panel shows the instant forcing due to the gases being present in
    the Earth's atmosphere (in watts per square meter),
- the middle panel shows the cumulative effect of that forcing, equal to
    the areas under the curves in the top graph integrated over the Earth's
    surface every year (in joules),
- the bottom panel shows the cumulated forcing (middle panel values) of
    gases relative to that of carbon dioxide. This measure is called "global
    warming potential" and is usually calculated for three time horizons: 20,
    100, and 500 years, which are shown on the bottom graph.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import trange
from matplotlib import animation

Radiative forcing is modelled as a linear combination of exponential decay functions:

$$
IRF_x = a_{x,0} + \sum a_{x,i} e^\frac{-t}{τ_x,i} \textrm{ for } t \geq 0
$$

with 

$$
\sum a_{x,i} = 1
$$

For most substances, $a_{x,0} = 0$, i.e. they disappear completely from the atmosphere given enough time. Moreover, the only non-zero coefficient is commonly $a_{x,1} = 1$ (except for CO<sub>2</sub>), which yields:

$$
IRF_x = \sum e^\frac{-t}{τ_x}
$$

The variable $τ_x$ is the lifetime of the substance.

In [5]:
a_x_i = pd.read_csv('GHG_property_table.csv',index_col=0)
a_CO2_i = pd.read_csv('CO2_property_table.csv')
a_x_i.iloc[1:,:5].head()

,Chemical Formula,Lifetime (Years),Radiative Efficiency (W m-2 ppb-1),AGWP 20-year (W m-2 yr kg-1),GWP 20-year
"Acronym, Common Name or Chemical Name",,,,,
Methane,CH4,12.4,0.000363,2.090000e-12,84
Fossil methane,CH4,12.4,0.000363,2.110000e-12,85
Nitrous oxide,N2O,121,0.003000,6.580000e-12,264
CFC-11,CCl3F,45,0.260000,1.720000e-10,6900
CFC-12,CCl2F2,100,0.320000,2.690000e-10,10800


In [4]:
GHG[['Lifetime (Years)','Radiative Efficiency (W m-2 ppb-1)']].head()

NameError: name 'GHG' is not defined

The radiative forcing of CO<sub>2</sub> in the atmosphere is modelled as the sum of a constant (i.e. a fraction of the emitted CO<sub>2</sub> remains forever, $\tau_0 = \infty$) and three exponential decay functions:

In [ ]:
CO2_a = CO2[['a'+str(i) for i in range(4)]].copy()
CO2_a['sum of a coefficients'] = CO2_a.sum(1)
CO2_a

In [ ]:
# Constants
substances = [
#       'Sulfur hexafluoride(Air/)',
       'Dinitrogen monoxide(Air/)',
       'Methane, fossil(Air/)',
#       'Ethane, 1,1-difluoro-, HFC-152a(Air/high. pop.)',
#       'Ethane, 1,1,1,2-tetrafluoro-, HFC-134a(Air/high. pop.)',
#       'Methane, trichlorofluoro-, CFC-11(Air/high. pop.)',
#       'Methane, tetrafluoro-, CFC-14(Air/high. pop.)',
       'Carbon dioxide(Air/)']

earth_area = 510072000000000 # m2
year_in_s  = 365.2425 * 24 * 3600 # s/yr

n_sub = len(substances)

In [ ]:
time = np.linspace(0,1000,10001)
IRF = pd.DataFrame(np.exp(np.outer(-time,1/GHG['Lifetime (Years)'])),
            index=time,
            columns=GHG.index)

IRF['Carbon dioxide'] = CO2.a0.values + \
                            CO2.a1.values * np.exp(np.outer(-time,1/CO2.т1)) + \
                            CO2.a2.values * np.exp(np.outer(-time,1/CO2.т2)) + \
                            CO2.a3.values * np.exp(np.outer(-time,1/CO2.т3))

In [ ]:
IRF.T.head().plot()